# Reporte de Calidad de Datos (DQR)
---

## ¿Que es la calidad de datos?

Es la cualidad de un conjunto de información recogida de un dataset que reúne entre sus atributos:

- Exactitud
- Completitud
- Integridad
- Actualización
- Coherencia
- Relevancia
- Accesibilidad
- Confiabilidad

Cada una necesarias para resultar útiles al procesamiento, análisis y cualquier otro fin que un usuario quiera darles.

## Librería de calidad de datos

La librería de calidad de datos tiene como objetivo principal ser una herramienta que facilite la verificación de contenido de bases de datos y dé métricas de calidad para que usuarios puedan decidir si sus bases de datos necesitan modificarse para ser utilizadas en los proyectos.

La librería fue escrita en el lenguaje de programación de Python y puede analizar bases de datos estructurados que se conviertan en objetos dataframe. Contiene tres módulos principales:
- El módulo Calidad datos para analizar cualquier base de datos
- El módulo Datos gov para conectarse con los metadatos del Portal de Datos Abiertos de Colombia utilizar sus bases de datos
- El módulo de Reporte el cual permite generar un reporte de calidad utilizando los módulos anteriores.


Instalamos la librería

In [4]:
!pip install leila

DEPRECATION: Python 2.7 reached the end of its life on January 1st, 2020. Please upgrade your Python as Python 2.7 is no longer maintained. pip 21.0 will drop support for Python 2.7 in January 2021. More details about Python 2 support in pip can be found at https://pip.pypa.io/en/latest/development/release-process/#python-2-support pip 21.0 will remove support for this functionality.
Defaulting to user installation because normal site-packages is not writeable
ERROR: Could not find a version that satisfies the requirement leila (from versions: none)
ERROR: No matching distribution found for leila


Los datos utilizados para los ejemplos corresponden a una muestra de los Casos positivos de COVID-19 en Colombia disponibles en el portal de [Datos Abiertos de Colombia](https://www.datos.gov.co/Salud-y-Protecci-n-Social/Casos-positivos-de-COVID-19-en-Colombia/gt2j-8ykr), se debe aclarar que los datos fueron modificados con el propósito de presentar el alcance de las diferentes funciones de la librería.

In [ ]:
import pandas as pd

dtypes = {'ID de caso': object,

          'Código DIVIPOLA': object,

          'Codigo departamento': object,

          'Codigo pais':object}

# Lectura de archivo de datos como dataframe
df_datos = pd.read_excel('dataset ejemplos.xlsx', dtype=dtypes)

### Ejemplos de Reporte

#### **reporte.generar_reporte**

Crea un reporte de calidad de datos en formato HTML utilizando las funciones disponibles en los otros módulos de la librería. Ver documentación reporte.generar_reporte()

**Parámetros:**


- **castNumero** – (bool) {True, False}. Valor por defecto: False. Indica si se desea convertir las columnas de tipos object y bool a float, de ser posible.
- **df** – (dataframe) base de datos de insumo para la generación del reporte de calidad de datos.
- **api_id** – (str) opcional - Identificación de la base de datos asociado con la API de Socrata (de Datos Abiertos).
- **token** – (str) opcional - Token de usuario de la API de Socrata (de Datos Abiertos).
- **titulo** – (str) valor por defecto: “Reporte perfilamiento”. Título del reporte a generar.
- **archivo** – (str) valor por defecto: “perfilamiento.html”. Ruta donde guardar el reporte.
___

**Devuelve:**


- archivo de reporte en formato HTML.
___

In [ ]:
# Se importan las funciones de la librería
from leila.reporte import generar_reporte

# Se invoca la función para generar el reporte de calidad de datos
generar_reporte(df=df_datos,
                titulo='Perfilamiento datos COVID-19 - Colombia',
                archivo='perfilamiento.html')

### Ejemplos de Calidad de datos

#### **calidad_datos.CantidadDuplicados**

Retorna el porcentaje/número de filas o columnas duplicadas (repetidas) en el dataframe.

**Parámetros:**


- **eje** – (int) {1, 0}, valor por defecto: 0. Si el valor es 1 la validación se realiza por columnas, si el valor es 0 la validación se realiza por filas.
- **numero** – (bool) {True, False}, valor por defecto: False. Si el valor es False el resultado se expresa como un cociente, si el valor es True el valor se expresa como una cantidad de registros (número entero).
___
**Devuelve:**

- (int o float) resultado de unicidad.
___

In [ ]:
from leila.calidad_datos import CalidadDatos

# Creamos un objeto tipo CalidadDatos con el dataframe de datos de interés
datos = CalidadDatos(df_datos)

In [ ]:
# Numero de filas duplicadas
repetidos_col_prop = datos.CantidadDuplicados(eje=0, numero=True)
print(repetidos_col_prop)

In [ ]:
# Proporción de columnas duplicadas
repetidos_col_prop = datos.CantidadDuplicados(eje=1, numero=False)
print(repetidos_col_prop)

De acuerdo a los resultados obtenidos vemos que la base de datos de ejemplo presenta 10 filas duplicadas y el 8.3% (2 de 24) de las columnas duplicadas.

#### **calidad_datos.CorrelacionCategoricas**

Genera una matriz de correlación entre las variables de tipo categóricas.

**Parámetros:**

- **metodo** – (str) {“phik”, “cramer”}, valor por defecto: “phik”. Medida de correlación a utilizar.
- **limite** – (float) (valor de 0 a 1) límite de referencia, se utiliza para determinar si las variables posiblemente son de tipo categóricas y ser incluidas en el análisis. Si el número de valores únicos por columna es mayor al número de registros limite, se considera que la variable no es categórica.
- **categoriasMaximas** – (int) (valor mayor a 0), indica el máximo número de categorías de una variable para que sea incluida en el análisis
- **variables** – (list) lista de nombres de las columnas separados por comas. Permite escoger las columnas de interés de análisis del dataframe
___

**Devuelve:**

dataframe con las correlaciones de las columnas de tipo categórica analizadas.
___

Calcular la correlación de variables tipo categóricas, en este caso utilizaremos el método de cramer y limitaremos el cálculo a las variables “Atención”, “Sexo”, “Tipo” y “Estado”.

In [ ]:
from leila.calidad_datos import CalidadDatos

# Creamos un objeto tipo CalidadDatos con el dataframe de datos de interés
datos = CalidadDatos(df_datos)

# Se invoca la función para calcular la correlación de variables categóricas
matriz_cramer = datos.CorrelacionCategoricas(metodo='cramer', variables=['Atención', 'Sexo', 'Tipo', 'Estado'])
print(matriz_cramer)

#### **calidad_datos.CorrelacionNumericas**

Genera una matriz de correlación entre las variables de tipo numérico.


**Parámetros:**

- **metodo** – (str) {“pearson”, “kendall”, “spearman”}, valor por defecto: “pearson”. Medida de correlación a utilizar.
- **variables** – (list) lista de nombres de las columnas separados por comas. Permite escoger las columnas de interés de análisis del dataframe
___

**Devuelve:**

dataframe con las correlaciones de las columnas de tipo numérico analizadas.
___

In [ ]:
from leila.calidad_datos import CalidadDatos

# Creamos un objeto tipo CalidadDatos con el dataframe de datos de interés
datos = CalidadDatos(df_datos)

# Se invoca la función para calcular la correlación de variables numéricas
corr_spearman = datos.CorrelacionNumericas(metodo="spearman")
print(corr_spearman)